In [ ]:
import numpy as np
import pandas as pd
import snowflake.connector
from datetime import datetime, timedelta
import requests
import io
from scipy.optimize import linprog
import math
import os
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value

In [ ]:
#Cargar y concatenar los archivos que existan en un solo dataframe
dataframes = []
try:
    if os.path.exists(r"C:\Users\juane\Downloads\PLAN_PANADERIA.csv"):
        df_plan_panaderia = pd.read_csv(r"C:\Users\juane\Downloads\PLAN_PANADERIA.csv")
        dataframes.append(df_plan_panaderia)
        print("PLAN_PANADERIA.csv cargado con éxito.")
    else:
        print("PLAN_PANADERIA.csv no encontrado.")

    if os.path.exists(r"C:\Users\juane\Downloads\PLAN_CONGELADOS.csv"):
        df_plan_congelados = pd.read_csv(r"C:\Users\juane\Downloads\PLAN_CONGELADOS.csv")
        dataframes.append(df_plan_congelados)
        print("PLAN_CONGELADOS.csv cargado con éxito.")
    else:
        print("PLAN_CONGELADOS.csv no encontrado.")

    if os.path.exists(r"C:\Users\juane\Downloads\PLAN_COMPRA_GENERAL.csv"):
        df_plan_compra_general = pd.read_csv(r"C:\Users\juane\Downloads\PLAN_COMPRA_GENERAL.csv")
        dataframes.append(df_plan_compra_general)
        print("PLAN_COMPRA_GENERAL.csv cargado con éxito.")
    else:
        print("PLAN_COMPRA_GENERAL.csv no encontrado.")
    
    if not dataframes:
        raise FileNotFoundError("Ningún archivo CSV de planes de compra está presente. Proceso detenido.")
    
    df_plan_externos = pd.concat(dataframes, ignore_index=True)
    df_plan_externos = df_plan_externos.reset_index(drop=True)
    print("Archivos concatenados exitosamente.")
    
except FileNotFoundError as e:
    print(e)
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

In [ ]:
#Dataframe que relaciona warehouseId, cityId y CityName
warehouse_city_mapping = pd.DataFrame({
    'Warehouseid': [49, 63, 66, 84, 107, 21, 25, 27, 29, 31, 48, 52, 54, 55, 68, 80, 87, 91, 92, 3, 4, 5, 6, 7, 108, 109, 2, 75, 79, 90, 93, 8, 9, 11, 12, 14, 16, 17, 18, 19, 20, 23, 24, 26, 28, 30, 33, 47, 50, 51, 53, 56, 58, 60, 61, 67, 69, 70, 72, 73, 78, 110, 103, 104, 106, 105],
    'CITY_ID': [3, 3, 3, 5, 5, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 35, 35, 35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 40, 42, 42, 43],
    'CITY_NAME': ['Barranquilla', 'Barranquilla', 'Barranquilla', 'Bucaramanga', 'Bucaramanga', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Medellín', 'Cartagena', 'Barranquilla', 'Cartagena', 'Cartagena', 'Cartagena', 'Santa Marta', 'Villavicencio', 'Cali', 'Cali', 'Cali', 'Cali', 'Cali', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Bogotá', 'Armenia', 'Pereira', 'Ibagué', 'Montería']
})

In [ ]:
#Alistar el dataframe para ser procesado
df_plan_externos["WAREHOUSE_PRODUCT_ID"]=df_plan_externos["Warehouseid"].astype(str)+"-"+df_plan_externos["SKU ID"].astype(str)
df_plan_externos["ADJUSTED_QUANTITY"] = df_plan_externos["Purchase Quantity for Plan Duration Editable"]
df_plan_externos["Variable"] = df_plan_externos.apply(lambda row: LpVariable(f"Q_{row['WAREHOUSE_PRODUCT_ID']}", lowBound=row["ADJUSTED_QUANTITY"], cat="Integer"), axis=1)
df_plan_externos["Average Forecasts for Next 7 Days"] = df_plan_externos["Average Forecasts for Next 7 Days"].replace(0, 0.001)
df_plan_externos = df_plan_externos.merge(warehouse_city_mapping, on='Warehouseid', how='left')
df_plan_externos = df_plan_externos.reset_index(drop=True)

In [ ]:
#Diccionario que contiene los multiplos para proveedor y producto
multiples_restrictions = {
    402: 24,
    188: 1,
    398: {17814: 12, 17816: 12, 17818: 12},
    359: 5,
    143: 1,
    200: 5,
    63: 1,
    132: 24,
    179: 1,
    387: 12,
    1: 15,
    87: 1,
    343: 15,
    69: 1,
    121: 1,
    305: 20,
    296: 1,
    193: 1,
    199: 1,
    397: 1,
    375: {17405: 18, 17407: 24, 19217: 25, 22696: 18},
    318: 10,
    160: 1,
    128: 1,
    310: 24,
    315: 1,
    434: {23158: 12, 23159: 12, 23160: 15},
    163: 1,
    379: 1,
    377: 1,
    365: 1,
    201: 24,
    383: 5,
    369: 5,
    414: 5,
    356: 1,
    4: 10,
    94: 1,
    155: 1,
    125: {6913: 12, 6939: 24, 6949: 24, 8568: 24, 11053: 24},
    384: {21306: 8, 17541: 48, 21305: 48, 21304: 48, 17542: 48},
    175: 1,
    336: 1,
    158: 1,
    404: 6,
    347: 1,
    339: 10,
    344: 48,
    167: 8,
    385: 1,
    112: 1,
    327: 1,
    388: {17631: 12, 17630: 10, 17629: 24},
    409: 1,
    67: 1,
    390: 1,
    325: 1,
    72: 24,
    187: 14,
    328: 48,
    6: 24,
    302: 1,
    153: 20,
    168: 10,
    116: 1,
    142: 1,
    129: 1,
    148: 1,
    406: 1,
    114: 12,
    46: 10,
    77: {15561: 5, 15128: 5, 15130: 5, 7759: 5, 11407: 5, 7753: 5, 15560: 2, 15126: 2},
    88: {2015: 24, 11042: 24, 1013: 24, 1014: 24, 9175: 12, 2014: 24},
    149: 24,
    202: 1,
    119: 20,
    107: 1,
    331: 1,
    348: 24,
    351: {16827: 12, 16860: 24, 16862: 12, 12435: 100, 16865: 100, 16864: 50, 16867: 24},
    144: 1,
    366: 6,
    191: {9504: 12, 13611: 72},
    101: 1,
    396: 1,
    189: {9293: 14, 12862: 18},
    185: 10,
    334: 1,
    381: 1,
    314: 1,
    89: 12,
    223: {17633: 160, 9586: 160, 13625: 160, 13956: 10, 9571: 72},
    166: 1,
    354: 1,
    355: 10,
    171: 24,
    126: 1,
    395: 1,
    135: 6,
    350: 6,
    214: 24,
    204: 5,
    424: 18,
    113: 1,
    122: 1,
    386: 1,
    349: {16703: 6, 16704: 6, 16707: 12, 16708: 12, 16709: 12, 16710: 12, 16711: 12, 16712: 8, 16713: 8, 16714: 8},
    120: 1,
    358: 5,
    437: 1,
    57: 1,
    208: 5,
    338: 8,
    301: {18489: 5, 17709: 6, 15085: 6, 20072: 6, 18478: 12, 18477: 12, 17711: 12, 17710: 12, 16679: 12, 16678: 12, 15084: 12, 15083: 12, 14761: 12, 14363: 12, 14360: 12, 20074: 12, 20073: 12, 18472: 14, 18471: 16, 18470: 16, 18469: 16, 18468: 16, 18467: 16, 18486: 18, 14762: 18, 18493: 24, 18483: 24, 18480: 24, 18479: 24, 16680: 24, 15086: 24, 18482: 52, 18494: 66}
}

In [ ]:
supplier_restrictions = {
    402: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 72,
            "max_quantity": 50000
        }
    ],
    188: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 20,
            "max_quantity": 50000
        }
    ],
    398: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 1,
            "max_quantity": 50000
        }
    ],
    359: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 5,
            "max_quantity": 50000
        }
    ],
    143: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 350000,
            "max_value": 10000000
        }
    ],
    200: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 40,
            "max_quantity": 50000
        }
    ],
    63: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 60000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 60000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 60000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 60000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 60000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 60000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 60000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 60000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 60000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 60000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 60000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 60000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 60000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 60000,
            "max_value": 10000000
        },
        {
            "type": "quantity",
            "level": "product",
            "SKU_ID": 9109,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "product",
            "SKU_ID": 12510,
            "min_quantity": 40,
            "max_quantity": 50000
        }
    ],
    132: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_quantity": 24,
            "max_quantity": 50000
        }
    ],
    179: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 48,
            "max_quantity": 50000
        }
    ],
    387: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 24,
            "max_quantity": 50000
        }
    ],
    1: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 30,
            "max_quantity": 50000
        }
    ],
    87: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_value": 600000,
            "max_value": 10000000
        }
    ],
    343: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 15,
            "max_quantity": 50000
        }
    ],
    69: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_value": 300000,
            "max_value": 10000000
        }
    ],
    121: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 1,
            "max_quantity": 50000
        }
    ],
    305: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 20,
            "max_quantity": 50000
        }
    ],
    296: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 1,
            "max_quantity": 50000
        }
    ],
    193: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 1,
            "max_quantity": 50000
        }
    ],
    199: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 30,
            "max_quantity": 50000
        }
    ],
    397: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 150000,
            "max_value": 10000000
        }
    ],
    375: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 1,
            "max_quantity": 50000
        }
    ],
    318: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 20,
            "max_quantity": 50000
        }
    ],
    160: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 1,
            "max_quantity": 50000
        }
    ],
    128: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 30,
            "max_quantity": 50000
        }
    ],
    310: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 24,
            "max_quantity": 50000
        }
    ],
    315: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 30,
            "max_quantity": 50000
        }
    ],
    434: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 1,
            "max_quantity": 50000
        }
    ],
    163: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 1,
            "max_quantity": 50000
        }
    ],
    379: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_value": 300000,
            "max_value": 10000000
        }
    ],
    377: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 30,
            "max_quantity": 50000
        }
    ],
    365: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 10,
            "max_quantity": 50000
        }
    ],
    201: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 24,
            "max_quantity": 50000
        }
    ],
    383: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 5,
            "max_quantity": 50000
        }
    ],
    369: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 5,
            "max_quantity": 50000
        }
    ],
    "414": [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 5,
            "max_quantity": 50000
        }
    ],
    "356": [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_value": 450000,
            "max_value": 10000000
        }
    ],
    4: [
        {
            "type": "quantity",
            "level": "city",
            "CITY_ID": 37,
            "min_quantity": 250,
            "max_quantity": 50000
        }
    ],
    94: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 350000,
            "max_value": 10000000
        }
    ],
    155: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_value": 350000,
            "max_value": 10000000
        }
    ],
    125: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 24,
            "max_quantity": 50000
        }
    ],
    175: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 1,
            "max_quantity": 50000
        }
    ],
    336: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 1,
            "max_quantity": 50000
        }
    ],
    158: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 1,
            "max_quantity": 50000
        }
    ],
    404: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 6,
            "max_quantity": 50000
        }
    ],
    347: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 12,
            "max_quantity": 50000
        }
    ],
    339: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 10,
            "max_quantity": 50000
        }
    ],
    344: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 48,
            "max_quantity": 50000
        }
    ],
    167: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 24,
            "max_quantity": 50000
        }
    ],
    385: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 12,
            "max_quantity": 50000
        }
    ],
    112: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_value": 500000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_value": 250000,
            "max_value": 10000000
        }
    ],
    327: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 30,
            "max_quantity": 50000
        }
    ],
    388: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_value": 600000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 600000,
            "max_value": 10000000
        }
    ],
    409: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 3,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 3,
            "max_quantity": 50000
        }
    ],
    67: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 200000,
            "max_value": 10000000
        }
    ],
    390: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_value": 300000,
            "max_value": 10000000
        }
    ],
    325: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 30,
            "max_quantity": 50000
        }
    ],
    72: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 72,
            "max_quantity": 50000
        }
    ],
    187: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 14,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 14,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 14,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 14,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 14,
            "max_quantity": 50000
        }
    ],
    328: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 48,
            "max_quantity": 50000
        }
    ],
    6: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 72,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 72,
            "max_quantity": 50000
        }
    ],
    302: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 160,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 60,
            "max_quantity": 50000
        }
    ],
    153: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 20,
            "max_quantity": 50000
        }
    ],
    168: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 30,
            "max_quantity": 50000
        }
    ],
    116: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 300000,
            "max_value": 10000000
        }
    ],
    142: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_value": 350000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_value": 350000,
            "max_value": 10000000
        }
    ],
    129: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 1,
            "max_quantity": 50000
        }
    ],
    148: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 300000,
            "max_value": 10000000
        }
    ],
    406: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 1,
            "max_quantity": 50000
        }
    ],
    114: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 12,
            "max_quantity": 50000
        }
    ],
    46: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 10,
            "max_quantity": 50000
        }
    ],
    77: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 80000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_value": 80000,
            "max_value": 10000000
        }
    ],
    88: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 1,
            "max_quantity": 50000
        }
    ],
    149: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 24,
            "max_quantity": 50000
        }
    ],
    202: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 30,
            "max_quantity": 50000
        }
    ],
    119: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 40,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 40,
            "max_quantity": 50000
        }
    ],
    107: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_value": 800000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 300000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_value": 300000,
            "max_value": 10000000
        }
    ],
    331: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 200000,
            "max_value": 10000000
        }
    ],
    348: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 24,
            "max_quantity": 50000
        }
    ],
    351: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 1,
            "max_quantity": 50000
        }
    ],
    144: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_value": 130000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_value": 130000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 130000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 130000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 130000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 130000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 130000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 130000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 130000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 130000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 130000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 130000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 130000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 130000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 130000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 130000,
            "max_value": 10000000
        }
    ],
    366: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 6,
            "max_quantity": 50000
        }
    ],
    191: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 1,
            "max_quantity": 50000
        }
    ],
    101: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_value": 150000,
            "max_value": 10000000
        }
    ],
    396: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 37,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 37,
            "max_quantity": 50000
        }
    ],
    189: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 1,
            "max_quantity": 50000
        }
    ],
    185: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 30,
            "max_quantity": 50000
        }
    ],
    334: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 60,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 60,
            "max_quantity": 50000
        }
    ],
    314: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 1,
            "max_quantity": 50000
        }
    ],
    89: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 36,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 36,
            "max_quantity": 50000
        }
    ],
    223: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_value": 400000,
            "max_value": 10000000
        }
    ],
    166: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 20,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 20,
            "max_quantity": 50000
        }
    ],
    354: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 12,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 12,
            "max_quantity": 50000
        }
    ],
    355: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 10,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 10,
            "max_quantity": 50000
        }
    ],
    171: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 48,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 48,
            "max_quantity": 50000
        }
    ],
    126: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 30,
            "max_quantity": 50000
        }
    ],
    395: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_value": 200000,
            "max_value": 10000000
        }
    ],
    135: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 18,
            "max_quantity": 50000
        }
    ],
    350: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 6,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 6,
            "max_quantity": 50000
        }
    ],
    214: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 24,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 24,
            "max_quantity": 50000
        }
    ],
    204: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 5,
            "max_quantity": 50000
        }
    ],
    424: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_quantity": 18,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 18,
            "max_quantity": 50000
        }
    ],
    113: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 150000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 150000,
            "max_value": 10000000
        }
    ],
    122: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 30,
            "max_quantity": 50000
        }
    ],
    386: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 15,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 15,
            "max_quantity": 50000
        }
    ],
    349: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 1,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 1,
            "max_quantity": 50000
        }
    ],
    120: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_value": 180000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_value": 180000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 180000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_value": 180000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 180000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 180000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 180000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 180000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_value": 180000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 180000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_value": 180000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 180000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 180000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 180000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 180000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 180000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_value": 180000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_value": 180000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 180000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 180000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_value": 400000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 180000,
            "max_value": 10000000
        }
    ],
    358: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 5,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 5,
            "max_quantity": 50000
        }
    ],
    437: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 30,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 30,
            "max_quantity": 50000
        }
    ],
    57: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 250000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 250000,
            "max_value": 10000000
        }
    ],
    208: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 25,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 25,
            "max_quantity": 50000
        }
    ],
    338: [
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_quantity": 16,
            "max_quantity": 50000
        },
        {
            "type": "quantity",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_quantity": 16,
            "max_quantity": 50000
        }
    ],
    301: [
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 93,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 25,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 90,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 84,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 6,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 33,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 3,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 27,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 68,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 24,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 2,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 79,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 16,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 14,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 28,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 17,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 5,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 7,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 47,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 9,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 51,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 48,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 19,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 21,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 23,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 72,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 103,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 29,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 73,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 18,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 11,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 49,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 8,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 55,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 110,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 66,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 67,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 12,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 104,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 31,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 60,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 70,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 4,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 58,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 107,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 105,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 50,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 52,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 63,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 78,
            "min_value": 200000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 109,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 106,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 108,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 91,
            "min_value": 450000,
            "max_value": 10000000
        },
        {
            "type": "monetary",
            "level": "warehouse",
            "WAREHOUSE_ID": 61,
            "min_value": 200000,
            "max_value": 10000000
        }
    ]
}

In [ ]:
#Funcion que redondea al multiplo mayor más cercano
def round_to_multiple(quantity, multiple):
    return round(quantity / multiple) * multiple

In [ ]:
#Funcion que recorre el diccionario de multiplos y filtra el dataframe de manera acorde para aplicar los multiplos que correspondan
def apply_multiples_restrictions(df, multiples_restrictions):  
    for supplier, restriction in multiples_restrictions.items():
        supplier_df = df[df['Primary Supplier Id'] == supplier]
        
        if isinstance(restriction, dict):
            #Proveedores con diferentes multiplos por producto
            for product, multiple in restriction.items():
                product_df = supplier_df[supplier_df['SKU ID'] == product]
                for idx, row in product_df.iterrows():
                    ADJUSTED_QUANTITY = round_to_multiple(row['ADJUSTED_QUANTITY'], multiple)
                    df.loc[idx, 'ADJUSTED_QUANTITY'] = ADJUSTED_QUANTITY
        else:
            #El mismo multiplo para todos los productos
            for idx, row in supplier_df.iterrows():
                ADJUSTED_QUANTITY = round_to_multiple(row['ADJUSTED_QUANTITY'], restriction)
                df.loc[idx, 'ADJUSTED_QUANTITY'] = ADJUSTED_QUANTITY
                
    return df

In [ ]:
#Funcion que optimiza las cantidades a comprar para que se ajusten a las restricciones
def optimize(df, restriction, restriction_type, objective):
    # Selección del modelo y objetivo según tipo de restricción y objetivo
    if restriction_type == 'quantity' and objective == "increment":
        model = LpProblem("Optimization_Plan", LpMinimize)
        model += lpSum([var * (1 / avg_forecast) for var, avg_forecast in zip(df["Variable"], df["Average Forecasts for Next 7 Days"])]), "Minimize_Increments"
        model += lpSum(df["Variable"]) >= restriction, "Sum_Adjusted_Quantity"
       # Restricción de vida útil para cada SKU
        for var, avg_forecast, shelf_life in zip(df["Variable"], df["Average Forecasts for Next 7 Days"], df["Shelf Life"]):
            model += (var * (1 / avg_forecast)) * 1.5 <= shelf_life, f"Shelf_Life_Constraint_{var.name}"
        model.solve()
        if LpStatus[model.status] == "Optimal":
            df["ADJUSTED_QUANTITY"] = [var.varValue for var in df["Variable"]]
    elif restriction_type == 'quantity' and objective == "decrease":
        model = LpProblem("Optimization_Plan", LpMaximize)
        model += lpSum([var * (1 / avg_forecast) for var, avg_forecast in zip(df["Variable"], df["Average Forecasts for Next 7 Days"])]), "Maximize_Increments"
        model += lpSum(df["Variable"]) <= restriction, "Sum_Adjusted_Quantity"
        model.solve()
        if LpStatus[model.status] == "Optimal":
            df["ADJUSTED_QUANTITY"] = [var.varValue for var in df["Variable"]]
    elif restriction_type == 'monetary' and objective == "increment":
        # Restricción de valor monetario con precios del proveedor
        model = LpProblem("Optimization_Plan", LpMinimize)
        model += lpSum([var * (1 / avg_forecast) for var, avg_forecast in zip(df["Variable"], df["Average Forecasts for Next 7 Days"])]), "Minimize_Increments"
        lpSum([var * price for var, price in zip(df["Variable"], df['Primary Supplier Price avg'])]) >= restriction, "Sum_Adjusted_Quantity"
        lpSum([var * price for var, price in zip(df["Variable"], df['Primary Supplier Price avg'])]) <= restriction * 1.1
        for var, avg_forecast, shelf_life in zip(df["Variable"], df["Average Forecasts for Next 7 Days"], df["Shelf Life"]):
            model += (var * (1 / avg_forecast)) * 1.5 <= shelf_life, f"Shelf_Life_Constraint_{var.name}"
        model.solve()
        if LpStatus[model.status] == "Optimal":
            df["ADJUSTED_QUANTITY"] = [var.varValue for var in df["Variable"]]
    elif restriction_type == 'monetary' and objective == "decrease":
        model = LpProblem("Optimization_Plan", LpMaximize)
        model += lpSum([var * (1 / avg_forecast) for var, avg_forecast in zip(df["Variable"], df["Average Forecasts for Next 7 Days"])]), "Maximize_Increments"
        # Restricción de valor monetario para disminuir
        lpSum([var * price for var, price in zip(df["Variable"], df['Primary Supplier Price avg'])]) <= restriction, "Sum_Adjusted_Quantity"
        model.solve()
        if LpStatus[model.status] == "Optimal":
            df["ADJUSTED_QUANTITY"] = [var.varValue for var in df["Variable"]]
    return df 

In [ ]:
#Levels: product,warehouse,city,total
#types: quantity, monetary
#Si en las restricciones no se especifica el nivel, se toma como total
df_plan_externos['VIOLATION'] = False
df_plan_externos['VIOLATION_DESCRIPTION'] = ''

for supplier, restrictions in supplier_restrictions.items():
    supplier_df = df_plan_externos[df_plan_externos['Primary Supplier Id'] == supplier]
    if supplier_df.empty:
        continue
    for restriction in restrictions:
        restriction_type = restriction['type']
        level = restriction['level']
        if level == 'warehouse':
            group_col = 'Warehouseid'
            restriction_key = restriction['WAREHOUSE_ID']
        elif level == 'city':
            group_col = 'CITY_ID'
            restriction_key = restriction['CITY_ID']
        elif level == 'product':
            group_col = 'SKU ID'
            restriction_key = restriction['SKU_ID']
        else: 
            group_col = None
        if group_col:
            if group_col not in supplier_df.columns:
                continue
            grouped = supplier_df.groupby(group_col)
            if not grouped.groups:
                continue
            if restriction_key in grouped.groups:
                group = grouped.get_group(restriction_key)
                if restriction_type == 'quantity':
                    total_quantity = group['ADJUSTED_QUANTITY'].sum()
                    if total_quantity < restriction['min_quantity'] or total_quantity > restriction['max_quantity']:
                        df_plan_externos.loc[group.index, 'VIOLATION'] = True
                        df_plan_externos.loc[group.index, 'VIOLATION_DESCRIPTION'] = f'{restriction_type} violation at {level} level for {group_col}={restriction_key}'
                        if total_quantity < restriction['min_quantity']:
                            min_quantity = restriction.get('min_quantity', 0)
                            adjusted_df = optimize(group, min_quantity, restriction_type, "increment")
                            df_plan_externos.loc[adjusted_df.index, 'ADJUSTED_QUANTITY']= adjusted_df['ADJUSTED_QUANTITY'].values
                        elif total_quantity > restriction['max_quantity']:
                            max_quantity = restriction.get('max_quantity', 0)
                            adjusted_df = optimize(group, max_quantity, restriction_type, "decrease")
                            df_plan_externos.loc[adjusted_df.index, 'ADJUSTED_QUANTITY']= adjusted_df['ADJUSTED_QUANTITY'].values
                elif restriction_type == 'monetary':
                    total_value = (group['ADJUSTED_QUANTITY'] * group['Primary Supplier Price avg']).sum()
                    if total_value < restriction['min_value'] or total_value > restriction['max_value']:
                        df_plan_externos.loc[group.index, 'VIOLATION'] = True
                        df_plan_externos.loc[group.index, 'VIOLATION_DESCRIPTION'] = f'{restriction_type} violation at {level} level for {group_col}={restriction_key}'
                        if total_value < restriction['min_value']:
                            min_value = restriction.get('min_value', 0)
                            adjusted_df = optimize(group, min_value, restriction_type, "increment")
                            df_plan_externos.loc[adjusted_df.index, 'ADJUSTED_QUANTITY']= adjusted_df['ADJUSTED_QUANTITY'].values
                        elif total_value > restriction['max_value']:
                            max_value = restriction.get('max_value', 0)
                            adjusted_df = optimize(group, max_value, restriction_type, "decrease")
                            df_plan_externos.loc[adjusted_df.index, 'ADJUSTED_QUANTITY']= adjusted_df['ADJUSTED_QUANTITY'].values
            else:
                print(f"{group_col} -> {restriction_key} is missing from the dataframe for supplier {supplier}")
                continue      
        #Si en las restricciones no se especifica el nivel, se toma como total
        else: 
            if restriction_type == 'quantity':
                total_quantity = supplier_df['ADJUSTED_QUANTITY'].sum()
                if total_quantity < restriction['min_quantity'] or total_quantity > restriction['max_quantity']:
                    df_plan_externos.loc[supplier_df.index, 'VIOLATION'] = True
                    df_plan_externos.loc[supplier_df.index, 'VIOLATION_DESCRIPTION'] = f'{restriction_type} violation at total level'
                    if total_quantity < restriction['min_quantity']:
                        min_quantity = restriction.get('min_quantity', 0)
                        adjusted_df = optimize(group, min_quantity, restriction_type, "increment")
                        df_plan_externos.loc[adjusted_df.index, 'ADJUSTED_QUANTITY']= adjusted_df['ADJUSTED_QUANTITY'].values
                    elif total_quantity > restriction['max_quantity']:
                        max_quantity = restriction.get('max_quantity', 0)
                        adjusted_df = optimize(group, max_quantity, restriction_type, "decrease")
                        df_plan_externos.loc[adjusted_df.index, 'ADJUSTED_QUANTITY']= adjusted_df['ADJUSTED_QUANTITY'].values
            elif restriction_type == 'monetary':
                total_value = (supplier_df['ADJUSTED_QUANTITY'] * supplier_df['Primary Supplier Price avg']).sum()
                if total_value < restriction['min_value'] or total_value > restriction['max_value']:
                    df_plan_externos.loc[supplier_df.index, 'VIOLATION'] = True
                    df_plan_externos.loc[supplier_df.index, 'VIOLATION_DESCRIPTION'] = f'{restriction_type} violation at total level'
                    if total_value < restriction['min_value']:
                            min_value = restriction.get('min_value', 0)
                            adjusted_df = optimize(group, min_value, restriction_type, "increment")
                            df_plan_externos.loc[adjusted_df.index, 'ADJUSTED_QUANTITY']= adjusted_df['ADJUSTED_QUANTITY'].values
                    elif total_value > restriction['max_value']:
                        max_value = restriction.get('max_value', 0)
                        adjusted_df = optimize(group, max_value, restriction_type, "decrease")
                        df_plan_externos.loc[adjusted_df.index, 'ADJUSTED_QUANTITY']= adjusted_df['ADJUSTED_QUANTITY'].values
                        
# Apply multiples restrictions and adjust quantities                    
df_plan_externos = apply_multiples_restrictions(df_plan_externos, multiples_restrictions)                        

In [ ]:
df_plan_externos['STATUS'] = 'Pass'
df_plan_externos['STATUS_DESCRIPTION'] = ''

for supplier, restrictions in supplier_restrictions.items():
    supplier_df = df_plan_externos[df_plan_externos['Primary Supplier Id'] == supplier]
    if supplier_df.empty:
        continue
    for restriction in restrictions:
        restriction_type = restriction['type']
        level = restriction['level']

        if level == 'warehouse':
            group_col = 'Warehouseid'
            restriction_key = restriction['WAREHOUSE_ID']
        elif level == 'city':
            group_col = 'CITY_ID'
            restriction_key = restriction['CITY_ID']
        elif level == 'product':
            group_col = 'SKU ID'
            restriction_key = restriction['SKU_ID']
        else: 
            group_col = None

        if group_col:
            if group_col not in supplier_df.columns:
                continue
            grouped = supplier_df.groupby(group_col)

            if not grouped.groups:
                continue

            if restriction_key in grouped.groups:
                group = grouped.get_group(restriction_key)
                if restriction_type == 'quantity':
                    total_quantity = group['ADJUSTED_QUANTITY'].sum()
                    if total_quantity < restriction['min_quantity'] or total_quantity > restriction['max_quantity']:
                        df_plan_externos.loc[group.index, 'STATUS'] = 'Fail'
                        df_plan_externos.loc[group.index, 'STATUS_DESCRIPTION'] = f'{restriction_type} violation at {level} level for {group_col}={restriction_key}'

                elif restriction_type == 'monetary':
                    total_value = (group['ADJUSTED_QUANTITY'] * group['Primary Supplier Price avg']).sum()
                    if total_value < restriction['min_value'] or total_value > restriction['max_value']:
                        df_plan_externos.loc[group.index, 'STATUS'] = 'Fail'
                        df_plan_externos.loc[group.index, 'STATUS_DESCRIPTION'] = f'{restriction_type} violation at {level} level for {group_col}={restriction_key}'
            else:
                continue
        else: 
            if restriction_type == 'quantity':
                total_quantity = supplier_df['ADJUSTED_QUANTITY'].sum()
                if total_quantity < restriction['min_quantity'] or total_quantity > restriction['max_quantity']:
                    df_plan_externos.loc[supplier_df.index, 'STATUS'] = 'Fail'
                    df_plan_externos.loc[supplier_df.index, 'STATUS_DESCRIPTION'] = f'{restriction_type} violation at total level'

            elif restriction_type == 'monetary':
                total_value = (supplier_df['ADJUSTED_QUANTITY'] * supplier_df['Primary Supplier Price avg']).sum()
                if total_value < restriction['min_value'] or total_value > restriction['max_value']:
                    df_plan_externos.loc[supplier_df.index, 'STATUS'] = 'Fail'
                    df_plan_externos.loc[supplier_df.index, 'STATUS_DESCRIPTION'] = f'{restriction_type} violation at total level'

In [ ]:
fecha_actual = datetime.now().strftime("%d-%m-%Y")
nombre_archivo = f"C:/Users/juane/Downloads/PLAN_EXTERNOS_COLOMBIA_{fecha_actual}.csv"
df_plan_externos.to_csv(nombre_archivo, index=False)

In [ ]:
df_wms_input_compra_general = pd.DataFrame()
df_wms_input_compra_general['WAREHOUSE_DESTINATION'] = df_plan_externos.loc[df_plan_externos['STATUS']== 'Pass', 'Warehouseid']
df_wms_input_compra_general['SUPPLIER_ID'] = df_plan_externos.loc[df_plan_externos['STATUS']== 'Pass', 'Primary Supplier Id']
df_wms_input_compra_general['RETAIL_ID'] = df_plan_externos.loc[df_plan_externos['STATUS']== 'Pass','SKU ID']
df_wms_input_compra_general['QUANTITY'] = df_plan_externos.loc[df_plan_externos['STATUS'] == 'Pass', 'ADJUSTED_QUANTITY'].astype(int).values
df_wms_input_compra_general['PLANNED_DATE'] = pd.to_datetime(datetime.now().date()) + pd.to_timedelta(df_plan_externos.loc[df_plan_externos['STATUS']== 'Pass','Primary Supplier Lead Time (Days)'], unit='d')
df_wms_input_compra_general['PLANNED_DATE'] = df_wms_input_compra_general['PLANNED_DATE'].dt.strftime('%d-%m-%Y')
df_wms_input_compra_general = df_wms_input_compra_general.loc[df_wms_input_compra_general['QUANTITY'] != 0].copy()
df_wms_input_compra_general = df_wms_input_compra_general.reset_index(drop=True)

In [ ]:
nombre_archivo = f"C:/Users/juane/Downloads/WMS_INPUT_PURCHASE_PLAN_{fecha_actual}.csv"
df_wms_input_compra_general.to_csv(nombre_archivo,index=False)

In [ ]:
for supplier_id in df_wms_input_compra_general['SUPPLIER_ID'].unique():
    supplier_df = df_wms_input_compra_general[df_wms_input_compra_general['SUPPLIER_ID'] == supplier_id]
    
    file_name = f"C:/Users/juane/Downloads/WMS_INPUT_PURCHASE_PLAN_{supplier_id}.csv"
    supplier_df.to_csv(file_name, index=False)